In [ ]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from urllib.parse import unquote
import pandas as pd
import time
import csv

In [ ]:
driver = webdriver.Edge()

In [ ]:
# 과목별 코드는 동일. url만 바꿔서 저장 

# 국어 
etoos_url = 'https://www.etoos.com/Lecture/Pop_Evaluate/List.asp?AREA_CD=0001'

# # 수학
# etoos_url = 'https://www.etoos.com/Lecture/Pop_Evaluate/List.asp?AREA_CD=0003'

# # 한국사
# etoos_url = 'https://www.etoos.com/Lecture/Pop_Evaluate/List.asp?AREA_CD=0016'

driver.get(etoos_url)

In [ ]:
ind = 1  # 수강후기 번호 (1~5)
no = 1  # 1~10페이지 중 위치한 곳 (1~9)
lecture_list = []  # 결과물이 저장되는 리스트

while 1:  
    try:
        # 후기 순차적으로 클릭
        driver.find_element(by='xpath', value=f'/html/body/form/div/div/div[3]/table/tbody/tr[{ind}]/td[4]/a/p[1]').click()
        time.sleep(1)

        main_info = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[1]/p').text
        temp_list = main_info.split('ㅣ')
        writer = temp_list[0].replace(' ', '').split('(')

        # 작성일 
        date = temp_list[1].replace(' ', '') 
        date_year = date.split('.')[0]

        if date_year == '2022':
            break  

        # 작성자 이름 
        name = writer[0]

        if len(temp_list) > 3: 
            # 나이
            age = writer[1].split(',')[0] 
        else:
            age = ''

        # 강의 제목 
        class_name = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[2]/div[2]/strong').text 
        # 선생님 이름 
        teacher_name = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[2]/div[2]/span').text.split(' ')[1]
        # 수강평 제목 
        title = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[1]/strong').text 

        # 강좌 및 교재 키워드 존재 확인 
        lecture_yn = driver.find_elements(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[2]/table/tbody/tr/td[1]/strong')

        if lecture_yn:
            # 강좌 구성 키워드
            lecture = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[2]/table/tbody/tr/td[1]/strong').text
            # 교재 및 자료 구성 키워드
            textbook = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[2]/table/tbody/tr/td[2]/strong').text

            # 키워드 존재 확인 
            keyword_yn = driver.find_elements(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[4]')

            if keyword_yn:
                # 전체 키워드 
                keyword = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[3]').text
                # 강의평내용
                contents = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[4]').text
                # 대상정보 
                target_list = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[3]/p').text.split('l')
                # 대상강의
                target = target_list[0].replace(' ', '')
                # 추천대상
                recommendation = target_list[1].replace(' ', '')
            else:
                # 강의평내용
                contents = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[3]').text

        else:
            keyword_yn = driver.find_elements(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[3]')

            if keyword_yn:
                # 전체 키워드
                keyword = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[2]').text
                # 강의평내용
                contents = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[3]').text

            else:
                # 강의평내용
                contents = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[2]/div[2]').text

 
        # 대상정보 
        target_list = driver.find_element(by='xpath', value='//*[@id="boxPopGuide"]/div/div[3]/div[3]/p').text.split('l')
        # 대상강의
        target = target_list[0].replace(' ', '')
        # 추천대상
        recommendation = target_list[1].replace(' ', '')
        
        lecture_list.append([class_name, teacher_name, title, name, age, date, lecture, textbook, keyword, contents, target, recommendation])

        driver.back()  # 뒤로가기
        time.sleep(1)

        ind += 1

        # 페이지 넘어가기
        if ind > 5:  
            ind = 1
            # 다음 페이지로 이동
            driver.find_element(by='xpath', value=f'/html/body/form/div/div/div[4]/span/a[{no}]').click()
            no += 1
            time.sleep(1)
            
            if no > 9:
                driver.find_element(by='xpath', value=f'/html/body/form/div/div/div[4]/span/a[10]/img').click()
                no = 1
                time.sleep(1)

    except Exception as e:
        print(f"예외 발생: {e}")
        break

In [ ]:
lecture_list = pd.DataFrame(lecture_list)
lecture_list.columns = ['강의제목', '선생님이름', '수강평제목', '작성자이름', '나이', '작성일', '강좌구성', '교재및자료구성', '키워드', '강의평내용', '대상강의', '추천대상']
lecture_list
lecture_list.to_csv('이투스_국어.csv')